In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 1_tng_analog_sample.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''First look at the IllustrisTNG analog sample.
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, glob, time, dill as pickle

## Matplotlib
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] == 'tng-dfs' or os.path.realpath(src_path) == '/':
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,'../../src/')
from tng_dfs import cutout as pcutout
from tng_dfs import io as pio
from tng_dfs import tree as ptree
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','RO','VO','ZO','LITTLE_H',
            'MW_MASS_RANGE']
data_dir,mw_analog_dir,ro,vo,zo,h,mw_mass_range = \
    putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=True)

# Figure path
fig_dir = './fig/tng_analog_sample/'
epsen_fig_dir = '/epsen_data/scr/lane/projects/tng-dfs/figs/notebooks/tng_analog_sample/'
os.makedirs(fig_dir,exist_ok=True)
os.makedirs(epsen_fig_dir,exist_ok=True)
show_plots = False

# Load tree data
# with open('../parse_sublink_trees/data/tree_primaries.pkl','rb') as handle:
#     tree_primaries = pickle.load(handle)
# with open('../parse_sublink_trees/data/tree_major_mergers.pkl','rb') as handle:
#     tree_major_mergers = pickle.load(handle)
# n_mw = len(tree_primaries)

### Fundamental properties

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

mass_stars = putil.mass_code_to_physical(mwsubs['mass_stars'])

ax.hist(mass_stars, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$M_{\star}$ [$M_{\odot}$]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

mass_dm = putil.mass_code_to_physical(mwsubs['mass_dm'])
print('minimum DM mass ',round(mass_dm.min()/1e12,4),
      '(e12 Msun) maximum DM mass ',round(mass_dm.max()/1e12,4),
      '(e12 Msun)')

ax.hist(mass_dm, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$M_{\rm DM}$ [$M_{\odot}$]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()

print('Least massive DM masses: ', np.sort(mass_dm)[:4]/1e12,4, 'e12 Msun')
print('Most massive DM masses: ', np.sort(mass_dm)[-4:]/1e12,4, 'e12 Msun')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

hmr_stars = putil.distance_code_to_physical(mwsubs['halfmassrad_stars'])

ax.hist(hmr_stars, bins=10, histtype='step', color='k', lw=2, density=False)

ax.set_xlabel(r'$R_{1/2,\star}$ [kpc]')
ax.set_ylabel(r'$N$')

fig.tight_layout()
fig.show()


## Download Sublink Trees

In [ ]:
force_download_tree = True
tree_dir = data_dir+'mw_analogs/sublink_trees/full/'
for i in range(len(mwsubs)):
    z0_sid = mwsubs[i]['id']
    sublink_filename = tree_dir+'sublink_'+str(z0_sid)+'.hdf5'
    if os.path.exists(sublink_filename) and not force_download_tree:
        continue
    print('Downloading sublink tree for subfind ID: '+str(z0_sid))
    putil.get( mwsubs[i]['trees:sublink'], directory=tree_dir, timeout=None )